In [1]:
# preparing data

### The RCNN consists of this major steps
1. Use selctive srach to propose Regions
2. Get the CNN features of this proopses regions unsig a pretrained CNN
3. Train a classifier on this CNN features for (K+1) classes, K is the number of object and 1 is the background class
    -  Prepare positive and negative sample to train the classifier
4. To reduce localization error train a regression model to predict the correct offset (BBox Ofset regressor)


In [1]:
import random
import os,sys
import numpy as np

In [18]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import os

In [3]:
parent_path=r'D:\LearningObjectDetection'
sys.path.append(parent_path)

In [4]:
import numpy as np
import pandas as pd
import cv2
import utils
print(np.__version__)
print(pd.__version__)

1.20.3
1.5.1


In [5]:
base_data_path=r"D:\LearningObjectDetection\data\raccoon_dataset"
base_ann_path=os.path.join(base_data_path,"data")
base_img_path=os.path.join(base_data_path,"images")
print(os.listdir(base_ann_path))

['raccoon_labels.csv', 'test.record', 'test_labels.csv', 'train.record', 'train_labels.csv']


In [6]:
train_df=pd.read_csv(os.path.join(base_ann_path,"train_labels.csv"))
test_df=pd.read_csv(os.path.join(base_ann_path,"test_labels.csv"))

print(train_df.head())
print(test_df.head())

         filename  width  height    class  xmin  ymin  xmax  ymax
0  raccoon-17.jpg    259     194  raccoon    95    60   167   118
1  raccoon-11.jpg    660     432  raccoon     3     1   461   431
2  raccoon-63.jpg    600     400  raccoon    74   107   280   290
3  raccoon-63.jpg    600     400  raccoon   227    93   403   298
4  raccoon-60.jpg    273     185  raccoon    58    33   197   127
          filename  width  height    class  xmin  ymin  xmax  ymax
0  raccoon-146.jpg    275     183  raccoon     4     4   271   180
1  raccoon-105.jpg    720     960  raccoon   250    49   714   869
2  raccoon-143.jpg    259     194  raccoon    17    29   238   162
3   raccoon-57.jpg    640     425  raccoon    82     6   638   423
4   raccoon-68.jpg    640     423  raccoon     1    24   517   423


In [6]:
img_name="raccoon-17.jpg"
img_path=os.path.join(base_img_path,img_name)
img=cv2.imread(img_path)
box_coord=train_df[train_df['filename']==img_name]
#print(box_coord)
x1,y1,x2,y2 = box_coord.xmin[0],box_coord.ymin[0],box_coord.xmax[0],box_coord.ymax[0]
print(x1,y1,x2,y2)
img_rect=cv2.rectangle(img.copy(),(x1,y1),(x2,y2),color=(255,0,0))
#cv2.imshow("img",img)
cv2.imshow("img_rect",img_rect)
cv2.waitKey(0)

95 60 167 118


-1

In [11]:
# our output class label directories
prepared_data_path = os.path.join(base_data_path, "prepared_data")
positive_path = os.path.join(*[prepared_data_path, "raccoon_test"])
negative_path = os.path.join(*[prepared_data_path, "no_raccoon_test"])
# loop over the output positive and negative directories
for dir_path in (positive_path, negative_path):
    # if the output directory does not exist yet, create it
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [14]:
# define the number of max proposals used when running selective
# search for (1) gathering training data and (2) performing inference
max_proposals = 2000
max_proposals_infer = 200

In [15]:
# define the maximum number of positive and negative images to be
# generated from each image
max_positive = 30
max_negative = 10

In [16]:
# defining positive an dnegatice ROIs

In [17]:
# first lest get our ROIs using selective search

In [18]:
# grab all image paths in the input images directory
img_names = os.listdir(base_img_path)
# initialize the total number of positive and negative images we have
# saved to disk so far


In [19]:
input_dims=(224,224)

In [20]:
total_positive = 0
total_negative = 0
# loop over the image paths
for (i, img_name) in enumerate(img_names):
    # show a progress report
    print("[INFO] processing image {}/{}...".format(i + 1,len(img_names)))
    img_path=os.path.join(base_img_path,img_name)
    #ann_df=train_df[train_df['filename']==img_name]
    ann_df=test_df[test_df['filename']==img_name]

    print(len(ann_df))
    gt_boxes=[]
    for idx,row in ann_df.iterrows():
        x1,y1,x2,y2=row.xmin,row.ymin,row.xmax,row.ymax
        #print(x1,y1,x2,y2)
        gt_boxes.append((x1,y1,x2,y2))
    # load the input image from disk
    image = cv2.imread(img_path)
    
    # run selective search on the image and initialize our list of
    # proposed boxes
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    rects = ss.process()
    proposed_rects= []
    # loop over the rectangles generated by selective search
    for (x, y, w, h) in rects:
        # convert our bounding boxes from (x, y, w, h) to (startX,
        # startY, startX, endY)
        proposed_rects.append((x, y, x + w, y + h))
        
    # initialize counters used to count the number of positive and
    # negative ROIs saved thus far
    positive_ROIs = 0
    negative_ROIs = 0
    # loop over the maximum number of region proposals
    for proposed_rect in proposed_rects[:max_proposals]:
        # unpack the proposed rectangle bounding box
        (prop_x1, prop_y1, prop_x2, prop_y2) = proposed_rect
        # loop over the ground-truth bounding boxes
        for gt_box in gt_boxes:
            # compute the intersection over union between the two
            # boxes and unpack the ground-truth bounding box
            iou = utils.IOU(gt_box, proposed_rect)
            (gt_x1, gt_y1, gt_x2,gt_y2) = gt_box
            # initialize the ROI and output path
            roi = None
            output_path = None
            
            # check to see if the IOU is greater than 70% *and* that
            # we have not hit our positive count limit
            if iou > 0.8 and positive_ROIs <= max_positive:
                # extract the ROI and then derive the output path to
                # the positive instance
                roi = image[prop_y1:prop_y2, prop_x1:prop_x2]
                filename = "{}.png".format(total_positive)
                outputPath = os.path.join(*[positive_path,filename])
                # increment the positive counters
                positive_ROIs += 1
                total_positive += 1
            # determine if the proposed bounding box falls *within*
            # the ground-truth bounding box
            fullOverlap = prop_x1 >= gt_x1
            fullOverlap = fullOverlap and prop_y1 >= gt_y1
            fullOverlap = fullOverlap and prop_x2 <= gt_y2
            fullOverlap = fullOverlap and prop_y2 <= gt_y2
            
            # check to see if there is not full overlap *and* the IoU
            # is less than 5% *and* we have not hit our negative
            # count limit
            if not fullOverlap and iou < 0.05 and negative_ROIs <= max_negative:
                # extract the ROI and then derive the output path to# the negative instance
                roi = image[prop_y1:prop_y2, prop_x1:prop_x2]
                filename = "{}.png".format(total_negative)
                outputPath = os.path.join(*[negative_path,filename])
                # increment the negative counters
                negative_ROIs += 1
                total_negative += 1
            # check to see if both the ROI and output path are valid
            if roi is not None and outputPath is not None:
                # resize the ROI to the input dimensions of the CNN
                # that we'll be fine-tuning, then write the ROI to# disk
                roi = cv2.resize(roi, input_dims,interpolation=cv2.INTER_CUBIC)
                cv2.imwrite(outputPath, roi)

[INFO] processing image 1/200...
0
[INFO] processing image 2/200...
0
[INFO] processing image 3/200...
0
[INFO] processing image 4/200...
0
[INFO] processing image 5/200...
0
[INFO] processing image 6/200...
0
[INFO] processing image 7/200...
0
[INFO] processing image 8/200...
1
[INFO] processing image 9/200...
1
[INFO] processing image 10/200...
0
[INFO] processing image 11/200...
0
[INFO] processing image 12/200...
0
[INFO] processing image 13/200...
0
[INFO] processing image 14/200...
1
[INFO] processing image 15/200...
0
[INFO] processing image 16/200...
0
[INFO] processing image 17/200...
0
[INFO] processing image 18/200...
0
[INFO] processing image 19/200...
0
[INFO] processing image 20/200...
0
[INFO] processing image 21/200...
0
[INFO] processing image 22/200...
0
[INFO] processing image 23/200...
0
[INFO] processing image 24/200...
0
[INFO] processing image 25/200...
0
[INFO] processing image 26/200...
0
[INFO] processing image 27/200...
0
[INFO] processing image 28/200...
0
[

In [ ]:
# clean the dataset manually as many negative samples are in the positive dataset

In [ ]:
# once we have dataset we will fine tune models to build racoon/no-raccoon classifier

In [7]:
lables_to_class={"raccoon":0,"no_raccoon":1}

In [8]:
data=[]
labels=[]

In [9]:
def get_data(raccoon_path,no_raccoon_path):
    data=[]
    labels=[]
    for dir_ in [raccoon_path,no_raccoon_path]:
        label=0
        if "no_raccoon" in dir_:
            label=1
        for img_name in os.listdir(dir_):
            
            img_path=os.path.join(dir_,img_name)
            img=cv2.imread(img_path)
            img=np.resize(img,(224,224))
            data.append(img)
            labels.append(label)
    return np.array(data),np.array(labels)

In [12]:
raccoon_tr_path = os.path.join(*[prepared_data_path, "raccoon"])
no_raccoon_tr_path = os.path.join(*[prepared_data_path, "no_raccoon"])
raccoon_te_path = os.path.join(*[prepared_data_path, "raccoon_test"])
no_raccoon_te_path = os.path.join(*[prepared_data_path, "no_raccoon_test"])

In [13]:
x_tr,y_tr=get_data(raccoon_tr_path ,no_raccoon_tr_path )
x_te,y_te=get_data(raccoon_te_path ,no_raccoon_te_path )

In [14]:
x_tr.shape

(2285, 224, 224)

In [15]:
x_te.shape

(590, 224, 224)

In [16]:
y_tr = to_categorical(y_tr)
y_te = to_categorical(y_te)


In [17]:
y_te

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [19]:
# construct the training image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

 303104/9406464 [..............................] - ETA: 10:43